# Bottle Processing
Author: Andrew Reed

### Motivation:
Independent verification of the suite of physical and chemical observations provided by OOI are critical for the observations to be of use for scientifically valid investigations. Consequently, CTD casts and Niskin water samples are made during deployment and recovery of OOI platforms, vehicles, and instrumentation. The water samples are subsequently analyzed by independent labs for  comparison with the OOI telemetered and recovered data.

However, currently the water sample data routinely collected and analyzed as part of the OOI program are not available in a standardized format which maps the different chemical analyses to the physical measurements taken at bottle closure. Our aim is to make these physical and chemical analyses of collected water samples available to the end-user in a standardized format for easy comprehension and use, while maintaining the source data files. 

### Approach:
Generating a summary of the water sample analyses involves preprocessing and concatenating multiple data sources, and accurately matching samples with each other. To do this, I first preprocess the ctd casts to generate bottle (.btl) files using the SeaBird vendor software following the SOP available on Alfresco. 

Next, the bottle files are parsed using python code and the data renamed following SeaBird's naming guide. This creates a series of individual cast summary (.sum) files. These files are then loaded into pandas dataframes, appended to each other, and exported as a csv file containing all of the bottle data in a single data file.

### Data Sources/Software:

* **sbe_name_map**: This is a spreadsheet which maps the short names generated by the SeaBird SBE DataProcessing Software to the associated full names. The name mapping originates from SeaBird's SBE DataProcessing support documentation.

* **Alfresco**: The Alfresco CMS for OOI at alfresco.oceanobservatories.org is the source of the ctd hex, xmlcon, and psa files necessary for generating the bottle files needed to create the sample summary sheet.

* **SBEDataProcessing-Win32**: SeaBird vendor software for processing the raw ctd files and generating the .btl files.


In [1]:
# Import packages used in this notebook
import os, sys, re
import pandas as pd
import numpy as np

In [2]:
# Load the name mapping for the column names
# Specifiy the local directory
sbe_name_map = pd.read_excel('/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')
# sbe_name_map = pd.read_excel('C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')

In [3]:
# Specify the local directory where the bottle (.btl) files are stored for a particular cruise
# dirpath = 'C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/ctd/'
dirpath = '/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/ctd/'

In [4]:
# Parse the data for the start_time
def parse_header(header):
    """
    Parse the header of bottle (.btl) files to get critical information
    for the summary spreadsheet.
    
    Args:
        header - an object containing the header of the bottle file as a list of
            strings, split at the newline.
    Returns:
        hdr - a dictionary object containing the start_time, filename, latitude,
            longitude, and cruise id.
    """
    hdr = {}
    for line in header:
        if 'start_time' in line.lower():
            start_time = pd.to_datetime(re.split('= |\[',line)[1])
            hdr.update({'Start Time':start_time.strftime('%Y-%m-%dT%H:%M:%SZ')})
        elif 'filename' in line.lower():
            hex_name = re.split('=',line)[1].strip()
            hdr.update({'Hex name':hex_name})
        elif 'latitude' in line.lower():
            start_lat = re.split('=',line)[1].strip()
            hdr.update({'Start Latitude':start_lat})
        elif 'longitude' in line.lower():
            start_lon = re.split('=',line)[1].strip()
            hdr.update({'Start Longitude':start_lon})
        elif 'cruise id' in line.lower():
            cruise_id = re.split(':',line)[1].strip()
            hdr.update({'Cruise ID':cruise_id})
        else:
            pass
    
    return hdr
        

In [11]:
# Now write a function to autopopulate the bottle summary sample sheet
sbe_name_map;
dirpath;

# Now 
files = [x for x in os.listdir(dirpath) if '.btl' in x]
for filename in files:
    filepath = os.path.abspath(dirpath+filename)
    
    # Load the raw content into memory
    with open(filepath) as file:
        content = file.readlines()
    content = [x.strip() for x in content]
    
    # Now parse the file content
    header = []
    columns = []
    data = []
    for line in content:
        if line.startswith('*') or line.startswith('#'):
            header.append(line)
        else:
            try:
                float(line[0])
                data.append(line)
            except:
                columns.append(line)
    
    # Parse the header
    hdr = parse_header(header)
    
    # Parse the column identifiers
    column_dict = {}
    for line in columns:
        for i,x in enumerate(line.split()):
            try:
                column_dict[i] = column_dict[i] + ' ' + x
            except:
                column_dict.update({i:x})
    
    # Parse the bottle data based on the column header locations
    data_dict = {x:[] for x in column_dict.keys()}

    for line in data:
        if line.endswith('(avg)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            for i,x in enumerate(values):
                data_dict[i].append(x)
        elif line.endswith('(sdev)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            data_dict[1].append(values[0])
        else:
            pass
            
    data_dict[1] = [' '.join(item) for item in zip(data_dict[1][::2],data_dict[1][1::2])]
    
    # With the parsed data and column names, match up the data and column
    # based on the location
    results = {}
    for key,item in column_dict.items():
        values = data_dict[key]
        results.update({item:values})
        
    # Put the results into a dataframe
    df = pd.DataFrame.from_dict(results)
        
    # Now add the parsed info from the header files into the dataframe
    for key,item in hdr.items():
        df[key] = item
        
    # Get the cast number
    cast = filename[filename.index('.')-3:filename.index('.')]
    df['Cast #'] = str(cast).zfill(3)
    
    # Generate a filename for the summary file
    outname = filename.split('.')[0] + '.sum'
    
    # Save the results
    df.to_csv(dirpath+outname)

In [12]:
# Now, for each "summary" file, load and append to each other
df = pd.DataFrame()
for file in os.listdir(dirpath):
    if '.sum' in file:
        df = df.append(pd.read_csv(dirpath+file))
    else:
        pass

In [13]:
sbe_name_map['Short Name'].apply(lambda x: str(x).lower());

In [14]:
# Rename the column title using the sbe_name_mapping 
for colname in list(df.columns.values):
    try:
        fullname = list(sbe_name_map[sbe_name_map['Short Name'].apply(lambda x: str(x).lower() == colname.lower()) == True]['Full Name'])[0]
        df.rename({colname:fullname},axis='columns',inplace=True)
    except:
        pass

In [15]:
df.columns.values

array(['Unnamed: 0', 'Bottle Position', 'Date Time',
       'Pressure, Digiquartz [db]', 'Depth [salt water, m]',
       'Latitude [deg]', 'Longitude [deg]', 'Temperature [ITS-90, deg C]',
       'Temperature, 2 [ITS-90, deg C]', 'Conductivity [S/m]',
       'Conductivity, 2 [S/m]', 'Salinity, Practical [PSU]',
       'Salinity, Practical, 2 [PSU]', 'Oxygen, SBE 43 [ml/l]',
       'Oxygen Saturation, Garcia & Gordon [ml/l]',
       'Beam Attenuation, WET Labs C-Star [1/m]',
       'Beam Transmission, WET Labs C-Star [%]', 'Hex name',
       'Start Latitude', 'Start Longitude', 'Cruise ID', 'Start Time',
       'Cast #'], dtype=object)

In [16]:
df.rename(columns={'Bottle Position':'Niskin #'},inplace=True)
df['Niskin #'] = df['Niskin #'].apply(lambda x: str( int(x) ) )
df.drop(columns='Unnamed: 0',inplace=True)
df['Cast #'] = df['Cast #'].apply(lambda x: str(x).zfill(3) )

In [ ]:
#df.to_csv(dirpath+'Irminger-5_Summary.csv')

### Oxygen & Salinity 
Now, we need to add the 

In [ ]:
# Need a way to map the oxygen and salinity samples to the CTD samples
# Load the CTD sample logs

In [18]:
os.listdir('Irminger/Irminger-5/Water Sampling/')

['Irminger-5_CTD_sampling_log.xlsx', 'Salts and O2']

In [35]:
botpath = '/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/Water Sampling/Irminger-5_CTD_sampling_log.xlsx'
sample_log = pd.read_excel(botpath,sheet_name='Summary',header=0)

In [36]:
sample_log.head()

,Cruise ID,Cast #,Lat,Lon,Bottom Depth,Date,Niskin #,Time,Target Depth,Trip Depth,...,Ph Bottle #,DIC/TA Bottle #,Salts Bottle #,Nitrate Bottle 1,Chlorophyll Brown Bottle #,Chlorophyll Filter Sample #,Unnamed: 19,Chlorophyll Brown Bottle Volume,Chlorophyll LN Tube,Comments
0,AR 30-03,001,62° 46.287' N,28° 39.405' W,1858 m,2018-06-06,1,09:26:00,150,147,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AR 30-03,001,62° 46.287' N,28° 39.405' W,1858 m,2018-06-06,2,09:31:00,100,105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AR 30-03,001,62° 46.287' N,28° 39.405' W,1858 m,2018-06-06,3,09:36:00,40,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AR 30-03,001,62° 46.287' N,28° 39.405' W,1858 m,2018-06-06,4,09:38:00,23,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AR 30-03,001,62° 46.287' N,28° 39.405' W,1858 m,2018-06-06,5,09:39:00,15,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# Rename the column headers
sample_log.rename(columns=lambda x: 'Sample Log: ' + x.strip(), inplace=True)
sample_log['Sample Log: Niskin #'] = sample_log['Sample Log: Niskin #'].apply(lambda x: int(x.replace('*','')) if type(x) == str else x )
sample_log['Sample Log: Niskin #'] = sample_log['Sample Log: Niskin #'].apply(lambda x: x if np.isnan(x) else str( int(x) ) )
sample_log['Sample Log: Cast #'] = sample_log['Sample Log: Cast #'].apply(lambda x: str(x).zfill(3))

In [38]:
set(list(sample_log['Sample Log: Niskin #']))

{'1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9'}

In [39]:
basepath = '/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/'

In [40]:
os.listdir(basepath+'Water Sampling/Salts and O2')

['003SAL.xlsx',
 '004SAL.xlsx',
 '005OXY.xlsx',
 '005SAL.xlsx',
 '006OXY.xlsx',
 '006SAL.xlsx',
 '007OXY.xlsx',
 '007SAL.xlsx',
 '010OXY.xlsx',
 '010SAL.xlsx',
 '011SAL.xlsx',
 '012SAL.xlsx',
 '014SAL.xlsx',
 '015OXY.xlsx',
 '015SAL.xlsx',
 '016SAL.xlsx',
 '017SAL.xlsx',
 '018SAL.xlsx',
 '019SAL.xlsx',
 '020SAL.xlsx']

In [41]:
# Load the oxygen data
sal_dir = basepath+'Water Sampling/Salts and O2/'
sal_df = pd.DataFrame()
oxy_df = pd.DataFrame()

for file in os.listdir(sal_dir):
    if 'SAL' in file:
        sal_df = sal_df.append(pd.read_excel(sal_dir+file))
    elif 'OXY' in file:
        oxy_df = oxy_df.append(pd.read_excel(sal_dir+file))

In [42]:
sal_df.rename(columns=lambda x: 'SAL: ' + x.strip(), inplace=True)
sal_df.dropna(subset=['SAL: Cruise'], inplace=True)
sal_df.fillna(-999, inplace=True)
sal_df['SAL: Station'] = sal_df['SAL: Station'].apply(lambda x: str( int(x) ).zfill(3) )
sal_df['SAL: Niskin #'] = sal_df['SAL: Niskin #'].apply(lambda x: str( int(x) ) )
sal_df['SAL: Sample'] = sal_df['SAL: Sample'].apply(lambda x: str( int(x) ) )
sal_df['SAL: Sample'] = sal_df['SAL: Case ID'] + sal_df['SAL: Sample']

In [43]:
oxy_df.rename(columns=lambda x: 'OXY: ' + x.strip(), inplace=True)
oxy_df.dropna(subset=['OXY: Cruise'], inplace=True)
oxy_df.fillna(-999, inplace=True)
oxy_df['OXY: Station'] = oxy_df['OXY: Station'].apply(lambda x: str( int(x) ).zfill(3) )
oxy_df['OXY: Niskin #'] = oxy_df['OXY: Niskin #'].apply(lambda x: str( int(x) ) )
oxy_df['OXY: Sample#'] = oxy_df['OXY: Sample#'].apply(lambda x: str( int(x) ) )
oxy_df['OXY: Sample#'] = oxy_df['OXY: Case'] + oxy_df['OXY: Sample#']

In [44]:
# Now I need to merge dataframes together
sample_log = sample_log.merge(sal_df, how='left', left_on=['Sample Log: Cast #','Sample Log: Salts Bottle #'], right_on=['SAL: Station','SAL: Sample'])

In [45]:
sample_log = sample_log.merge(oxy_df, how='left', left_on=['Sample Log: Cast #','Sample Log: Oxygen Bottle #'],
                              right_on=['OXY: Station','OXY: Sample#'])

In [ ]:
os.listdir('Irminger/Irminger-5/Water Sampling/')

In [ ]:
nutrients = pd.read_excel('Irminger/Irminger-5/Water Sampling/Irminger_Sea-04_AR21_Nutrients_Sample_Data_2017-09-20_ver_1-00.xlsx',header=0)

In [ ]:
nutrient_bottles = sample_log['Sample Log: Nitrate Bottle 1'].str.split('\n').apply(pd.Series, 1).stack()
nutrient_bottles.index = nutrient_bottles.index.droplevel(-1)
nutrient_bottles.name = 'Nitrate Bottle #'

In [ ]:
# Add the nutrient bottle number back into the sample log, and remove the excess '.'
sample_log = sample_log.join(nutrient_bottles)
sample_log['Nitrate Bottle #'] = sample_log['Nitrate Bottle #'].apply(lambda x: x.replace('.','') if type(x) == str else x)

In [ ]:
# Now I can add the nutrient bottle data to the sample log before loading
sample_log = sample_log.merge(nutrients, how='left', left_on='Nitrate Bottle #', right_on='Sample ID')

In [47]:
sample_columns = ['Sample Log: Cast #','Sample Log: Niskin #','SAL: Salinity','OXY: Oxy']#'Avg: Nitrate + Nitrite [µmol/L]', 'Avg: Ammonium [µmol/L]',
#      'Avg: Phosphate [µmol/L]', 'Avg: Silicate [µmol/L]', 'Avg: Nitrite [µmol/L]', 'Avg: Nitrate [µmol/L]']

In [48]:
result = df.merge(sample_log[sample_columns], how='left',
                  left_on=['Cast #','Niskin #'], right_on=['Sample Log: Cast #','Sample Log: Niskin #'])

In [49]:
result.columns.values

array(['Niskin #', 'Date Time', 'Pressure, Digiquartz [db]',
       'Depth [salt water, m]', 'Latitude [deg]', 'Longitude [deg]',
       'Temperature [ITS-90, deg C]', 'Temperature, 2 [ITS-90, deg C]',
       'Conductivity [S/m]', 'Conductivity, 2 [S/m]',
       'Salinity, Practical [PSU]', 'Salinity, Practical, 2 [PSU]',
       'Oxygen, SBE 43 [ml/l]',
       'Oxygen Saturation, Garcia & Gordon [ml/l]',
       'Beam Attenuation, WET Labs C-Star [1/m]',
       'Beam Transmission, WET Labs C-Star [%]', 'Hex name',
       'Start Latitude', 'Start Longitude', 'Cruise ID', 'Start Time',
       'Cast #', 'Sample Log: Cast #', 'Sample Log: Niskin #',
       'SAL: Salinity', 'OXY: Oxy'], dtype=object)

In [50]:
sal_unit = '[' + list(set(sal_df['SAL: Unit']) )[0] + ']'
oxy_unit = '[' + list(set(oxy_df['OXY: Unit']) )[0] + ']'
sal_unit, oxy_unit

('[psu]', '[mL/L]')

In [51]:
result.drop(['Sample Log: Cast #','Sample Log: Niskin #'], axis=1, inplace=True)

In [52]:
result.rename(columns=lambda x: x.replace('SAL:','Bottle') + ' ' + sal_unit if 'SAL:' in x else x, inplace=True)
result.rename(columns=lambda x: x.replace('OXY:','Bottle') + ' ' + oxy_unit if 'OXY:' in x else x, inplace=True)
result.rename(columns=lambda x: x.replace('Avg:','Bottle') if 'Avg:' in x else x, inplace=True)

In [53]:
result

,Niskin #,Date Time,"Pressure, Digiquartz [db]","Depth [salt water, m]",Latitude [deg],Longitude [deg],"Temperature [ITS-90, deg C]","Temperature, 2 [ITS-90, deg C]",Conductivity [S/m],"Conductivity, 2 [S/m]",...,"Beam Attenuation, WET Labs C-Star [1/m]","Beam Transmission, WET Labs C-Star [%]",Hex name,Start Latitude,Start Longitude,Cruise ID,Start Time,Cast #,Bottle Salinity [psu],Bottle Oxy [mL/L]
0,1,Jun 06 2018 09:26:42,148.209,146.727,62.77390,-28.65319,6.1764,6.1704,3.455169,3.454611,...,19.0368,0.8578 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,001,NaN,NaN
1,2,Jun 06 2018 09:31:25,105.773,104.726,62.77408,-28.65296,6.4277,6.4273,3.476623,3.476591,...,19.1611,0.8311 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,001,NaN,NaN
2,3,Jun 06 2018 09:36:05,40.563,40.167,62.77418,-28.65280,6.7659,6.7668,3.505544,3.505613,...,21.4223,0.4769 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,001,NaN,NaN
3,4,Jun 06 2018 09:38:14,23.395,23.168,62.77420,-28.65278,7.1446,7.1685,3.540467,3.542229,...,26.6674,0.1282 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,001,NaN,NaN
4,5,Jun 06 2018 09:39:31,15.493,15.343,62.77426,-28.65268,7.3552,7.3551,3.558928,3.558911,...,28.1697,0.1376 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,001,NaN,NaN
5,6,Jun 06 2018 09:41:34,2.739,2.713,62.77432,-28.65258,7.3695,7.3698,3.559666,3.559626,...,22.2056,0.3952 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,001,NaN,NaN
6,7,Jun 06 2018 09:41:42,2.972,2.943,62.77432,-28.65257,7.3710,7.3712,3.559819,3.559834,...,22.1383,0.3963 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,001,NaN,NaN
7,1,Jun 06 2018 14:24:52,155.360,153.807,62.50420,-29.79834,6.5441,6.5448,3.491486,3.491553,...,16.6596,1.5555 (avg),D:\Data\ar30-03002b.hex,62 30.13 N,029 47.90 W,ar30-03,2018-06-06T14:12:41Z,02b,NaN,NaN
8,2,Jun 06 2018 14:27:57,99.927,98.942,62.50471,-29.79834,6.6319,6.6321,3.496320,3.496352,...,17.7897,1.1718 (avg),D:\Data\ar30-03002b.hex,62 30.13 N,029 47.90 W,ar30-03,2018-06-06T14:12:41Z,02b,NaN,NaN
9,3,Jun 06 2018 14:31:07,39.452,39.068,62.50523,-29.79833,6.8601,6.8605,3.514970,3.515027,...,17.3674,1.3014 (avg),D:\Data\ar30-03002b.hex,62 30.13 N,029 47.90 W,ar30-03,2018-06-06T14:12:41Z,02b,NaN,NaN


In [54]:
# Replace all of the nans with -999999 and save to a csv
result.fillna(str(-9999999),inplace=True)

In [55]:
result

,Niskin #,Date Time,"Pressure, Digiquartz [db]","Depth [salt water, m]",Latitude [deg],Longitude [deg],"Temperature [ITS-90, deg C]","Temperature, 2 [ITS-90, deg C]",Conductivity [S/m],"Conductivity, 2 [S/m]",...,"Beam Attenuation, WET Labs C-Star [1/m]","Beam Transmission, WET Labs C-Star [%]",Hex name,Start Latitude,Start Longitude,Cruise ID,Start Time,Cast #,Bottle Salinity [psu],Bottle Oxy [mL/L]
0,1,Jun 06 2018 09:26:42,148.209,146.727,62.77390,-28.65319,6.1764,6.1704,3.455169,3.454611,...,19.0368,0.8578 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,001,-9999999,-9999999
1,2,Jun 06 2018 09:31:25,105.773,104.726,62.77408,-28.65296,6.4277,6.4273,3.476623,3.476591,...,19.1611,0.8311 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,001,-9999999,-9999999
2,3,Jun 06 2018 09:36:05,40.563,40.167,62.77418,-28.65280,6.7659,6.7668,3.505544,3.505613,...,21.4223,0.4769 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,001,-9999999,-9999999
3,4,Jun 06 2018 09:38:14,23.395,23.168,62.77420,-28.65278,7.1446,7.1685,3.540467,3.542229,...,26.6674,0.1282 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,001,-9999999,-9999999
4,5,Jun 06 2018 09:39:31,15.493,15.343,62.77426,-28.65268,7.3552,7.3551,3.558928,3.558911,...,28.1697,0.1376 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,001,-9999999,-9999999
5,6,Jun 06 2018 09:41:34,2.739,2.713,62.77432,-28.65258,7.3695,7.3698,3.559666,3.559626,...,22.2056,0.3952 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,001,-9999999,-9999999
6,7,Jun 06 2018 09:41:42,2.972,2.943,62.77432,-28.65257,7.3710,7.3712,3.559819,3.559834,...,22.1383,0.3963 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,001,-9999999,-9999999
7,1,Jun 06 2018 14:24:52,155.360,153.807,62.50420,-29.79834,6.5441,6.5448,3.491486,3.491553,...,16.6596,1.5555 (avg),D:\Data\ar30-03002b.hex,62 30.13 N,029 47.90 W,ar30-03,2018-06-06T14:12:41Z,02b,-9999999,-9999999
8,2,Jun 06 2018 14:27:57,99.927,98.942,62.50471,-29.79834,6.6319,6.6321,3.496320,3.496352,...,17.7897,1.1718 (avg),D:\Data\ar30-03002b.hex,62 30.13 N,029 47.90 W,ar30-03,2018-06-06T14:12:41Z,02b,-9999999,-9999999
9,3,Jun 06 2018 14:31:07,39.452,39.068,62.50523,-29.79833,6.8601,6.8605,3.514970,3.515027,...,17.3674,1.3014 (avg),D:\Data\ar30-03002b.hex,62 30.13 N,029 47.90 W,ar30-03,2018-06-06T14:12:41Z,02b,-9999999,-9999999


In [56]:
result.to_csv(basepath+'Irminger-5_Summary.csv')

In [57]:
os.getcwd()

'/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data'